In [ ]:
from __future__ import absolute_import, division, print_function

from hippylib import nb
import dolfin as dl
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
%matplotlib inline

import logging
logging.getLogger('FFC').setLevel(logging.ERROR)
logging.getLogger('UFC').setLevel(logging.ERROR)
dl.set_log_active(False)

sys.path.insert(0,'/home/fenics/Installations/MUQ_INSTALL/lib')
from IPython.display import Image

# MUQ Includes
import pymuqModeling as mm # Needed for Gaussian distribution
import pymuqApproximation as ma # Needed for Gaussian processes
import pymuqSamplingAlgorithms as ms # Needed for MCMC

from Optical_forward import Optical_forward

# Hyperparameters

In [ ]:
nx = ny = 20
mesh = dl.UnitSquareMesh(nx, ny)
V = dl.FunctionSpace(mesh, 'Lagrange', 1)

#sigma_true = dl.interpolate(dl.Expression('0.3 + 0.2 * sin(4 * pi * x[0]) * sin(4 * pi * x[1])', degree=5),V)
sigma_true = dl.interpolate(dl.Expression('0.1 + 0.1*(pow(pow(x[0] - 0.5,2) + pow(x[1] - 0.5,2),0.5) < 0.1)', degree=5),V)
#mu_true = dl.interpolate(dl.Expression('0.1 + 0.1*(pow(pow(x[0] - 0.5,2) + pow(x[1] - 0.75,2),0.5) < 0.1) + 0.2*(pow(pow(x[0] - 0.5,2) + pow(x[1] - 0.25,2),0.5) < 0.07)', degree=5),V)
gamma = dl.interpolate(dl.Expression('0.03 + 0.01 * sin(2 * pi * x[1])', degree=3), V)
Gamma = dl.Constant("1.0")
fwdSolver = Optical_forward(V, gamma, Gamma)
H = dl.Function(V)
nb.plot(sigma_true)
plt.title("sigma true")
plt.show()

# nb.plot(mu_true)
# plt.title("mu true")
# plt.show()

#H.vector().set_local(fwdSolver.Evaluate([sigma_true.vector().get_local(), mu_true.vector().get_local()])[0])
H.vector().set_local(fwdSolver.Evaluate([sigma_true.vector().get_local()])[0])

# Prior

In [ ]:
xDim=2;
yDim=1;

var = 1    # Marginal Variance
length = 1 # Lengthscale of the kernel
nu = 1.0/2.0 # Smoothness parameter

mean_sigma = ma.LinearMean(np.zeros((1,xDim)), [0.2])
kern_sigma = ma.MaternKernel(xDim, var, length, nu)

#mean_mu = ma.LinearMean(np.zeros((1,xDim)), [0.1])
#kern_mu = ma.MaternKernel(xDim, var, length, nu)

prior_sigma = ma.GaussianProcess(mean_sigma, kern_sigma).Discretize(mesh.coordinates().T).AsDensity()
#prior_mu = ma.GaussianProcess(mean_mu, kern_mu).Discretize(mesh.coordinates().T).AsDensity()

# Likelihood

In [ ]:
noiseVar = 1e-4
noiseCov = noiseVar*np.ones((V.dim()))
likelihood = mm.Gaussian(H.vector().get_local(), noiseCov).AsDensity()

# Posterior

In [ ]:
#posteriorPiece = mm.DensityProduct(3)
posteriorPiece = mm.DensityProduct(2)
sigma = mm.IdentityOperator(V.dim())
# mu = mm.IdentityOperator(V.dim())

# Work Graph

In [ ]:
graph = mm.WorkGraph()

# Forward model nodes and edges
graph.AddNode(sigma, "sigma")
# graph.AddNode(mu, "mu")
#graph.AddNode(obsOperator, "B")
graph.AddNode(fwdSolver, "u")

graph.AddEdge("sigma", 0, "u", 0)
# graph.AddEdge("mu", 0, "u", 1)
#graph.AddEdge("u", 0, "B", 0)

# Other nodes and edges
graph.AddNode(likelihood, "Likelihood")
graph.AddNode(prior_sigma, "Prior Sigma")
#graph.AddNode(prior_mu, "Prior Mu")
graph.AddNode(posteriorPiece,"Posterior")

#graph.AddEdge("B", 0, "Likelihood", 0)
graph.AddEdge("u", 0, "Likelihood", 0)
graph.AddEdge("sigma", 0, "Prior Sigma", 0)
#graph.AddEdge("mu", 0, "Prior Mu", 0)
graph.AddEdge("Prior Sigma",0,"Posterior",0)
#graph.AddEdge("Prior Mu",0,"Posterior",1)
graph.AddEdge("Likelihood",0, "Posterior",1)

In [ ]:
graph.Visualize("PosteriorGraph.png")
Image(filename='PosteriorGraph.png') 

In [ ]:
likelihood = graph.CreateModPiece("Likelihood")
start_sigma = sigma_true.vector().get_local() #2*np.ones(V.dim())
print("Likelihood: {}".format(likelihood.Evaluate([start_sigma])[0]))

pr = graph.CreateModPiece("Prior Sigma")
print("Prior Probability: {}".format(pr.Evaluate([start_sigma])[0]))

post= graph.CreateModPiece("Posterior")
print("Posterior Probability: {}".format(post.Evaluate([start_sigma])[0]))


In [ ]:
problem = ms.SamplingProblem(graph.CreateModPiece("Posterior"))

In [ ]:
# proposalOptions = dict()
# proposalOptions['Method'] = 'MHProposal'
# proposalOptions['ProposalVariance'] = 1e-2

# Setup pCN sampler
proposalOptions = dict()
proposalOptions['Method'] = 'CrankNicolsonProposal'
proposalOptions['Beta'] = 0.002
proposalOptions['PriorNode'] = 'Prior Sigma'

kernelOptions = dict()
kernelOptions['Method'] = 'MHKernel'
kernelOptions['Proposal'] = 'ProposalBlock'
kernelOptions['ProposalBlock'] = proposalOptions

options = dict()
options['NumSamples'] = 15000
options['ThinIncrement'] = 1
options['BurnIn'] = 100
options['KernelList'] = 'Kernel1'
options['PrintLevel'] = 3
options['Kernel1'] = kernelOptions

mcmc = ms.SingleChainMCMC(options,problem)

In [ ]:
start_sigma = 0.2*np.ones(V.dim()) #sigma_true.vector().get_local()
#start_mu = mu_true.vector().get_local() #0.1*np.ones(V.dim())
#samps = mcmc.Run([start_sigma, start_mu])
samps = mcmc.Run([start_sigma])

In [ ]:
sampMean = samps.Mean()
sampCov = samps.Covariance()
sampStd = np.sqrt(np.diag(sampCov))
#ess = samps.ESS()
#mcErr = np.sqrt( samps.Variance() / ess)

nb.plot(sigma_true)
plt.title("True sigma")
plt.savefig("truesigma_optical.png")
plt.show()
# nb.plot(mu_true)
# plt.title("True mu")
# plt.show()

sigma_post = dl.Function(V)
sigma_post.vector().set_local(sampMean[:V.dim()])
nb.plot(sigma_post)
plt.title("Posterior mean sigma")
plt.savefig("sigma_posterior.png")
plt.show()

sigma_post_std = dl.Function(V)
sigma_post_std.vector().set_local(sampStd)
nb.plot(sigma_post_std)
plt.title("Posterior Std. Deviation")
plt.savefig("sigma_post_std.png")
plt.show()

# mu_post = dl.Function(V)
# mu_post.vector().set_local(sampMean[V.dim():])
# nb.plot(mu_post)
# plt.title("Posterior mean mu")
# plt.show()

In [ ]:
sampMat = samps.AsMatrix()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(sampMat.T[:,:])
plt.savefig("sigma_mcmc.png")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))

for i in range(V.dim()):
    shiftedSamp = sampMat[i,:]-np.mean(sampMat[i,:])
    corr = np.correlate(shiftedSamp, shiftedSamp, mode='full')
    plt.plot(corr[int(corr.size/2):]/np.max(corr), label='Dimension %d'%i)
    
maxLagPlot = 15000
plt.plot([-maxLagPlot,0.0],[4.0*maxLagPlot,0.0],'--k', label='Zero')

plt.xlim([0,maxLagPlot])
plt.ylim([-0.1,1.1])
plt.title("Autocorrelation of sigma chains")
#plt.legend()
plt.savefig("autocorr_sigma.png")
plt.show()